In [1]:
#Initial imports 

import os
import pandas as pd
import requests
from requests import get
from datetime import datetime
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
## json R=responses ##

sid = SentimentIntensityAnalyzer()
query="Robinhood"
url = f"https://api.pushshift.io/reddit/search/comment/?q={query}"
request = requests.get(url)
json_response = request.json()

In [3]:

##  SubReddits are pulled from pushfit url f"https://api.pushshift.io/reddit/search/comment/?q={query}" ##

 ## setting pushfit url 
    
def get_pushshift_data(data_type, **kwargs):

    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
    payload = kwargs
    request = requests.get(base_url, params=payload)
    return request.json()

In [4]:
##  pushfit parameters 

data_type="comment"    
query="Wall Street Bets"         
duration="1460d"        
size=500               
sort_type="score"       
sort="desc"             
aggs="subreddit"

In [5]:
# loop funtion to pull subreddit metadata

data_all = []

for req in range(0, 47):
    data = get_pushshift_data(data_type="submission",
                          q="Wall Street Bets",
                          after=duration,
                          limit=499,
                          metadata=True)
    
    for item in data['data']:
        data_all.append(item)
        
    duration = data['data'][-1]['created_utc']+2629743
    if req % 4 == 0:
        print(req)

0
4
8
12
16
20


IndexError: list index out of range

In [ ]:
data_all[0]

In [ ]:
requests.get

In [6]:
len(data['data'])
data['data'][-1]

IndexError: list index out of range

In [7]:
data['metadata']

{'after': 1617541950,
 'agg_size': 100,
 'api_version': '3.0',
 'before': None,
 'es_query': {'query': {'bool': {'filter': {'bool': {'must': [{'range': {'created_utc': {'gt': 1617541950}}},
       {'simple_query_string': {'default_operator': 'and',
         'fields': ['title^5', 'selftext^2'],
         'query': 'Wall Street Bets'}}],
      'should': []}},
    'must_not': []}},
  'size': 100,
  'sort': {'created_utc': 'asc'}},
 'execution_time_milliseconds': 15.16,
 'index': 'rs_deltae',
 'limit': '499',
 'metadata': 'True',
 'q': 'Wall Street Bets',
 'ranges': [{'range': {'created_utc': {'gt': 1617541950}}}],
 'results_returned': 0,
 'shards': {'failed': 0, 'skipped': 0, 'successful': 3, 'total': 4},
 'size': 100,
 'sort': 'asc',
 'sort_type': 'created_utc',
 'timed_out': False,
 'total_results': 0}

In [8]:
df = pd.DataFrame(data_all)
df.head()

,author,author_created_utc,author_flair_css_class,author_flair_text,author_fullname,brand_safe,contest_mode,created_utc,domain,full_link,...,domain_override,events,eventsOnRender,href_url,is_blank,mobile_ad_url,outbound_link,promoted,show_media,third_party_trackers
0,chafien,1.416930e+09,None,None,t2_jljl6,False,False,1488982363,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Generalparks,1.384396e+09,OC-CC,,t2_dwnmj,False,False,1488989677,self.OCLions,https://www.reddit.com/r/OCLions/comments/5y8s...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Mo0se69,1.452988e+09,None,None,t2_tuz6t,True,False,1488996369,self.stocks,https://www.reddit.com/r/stocks/comments/5y9ho...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RissaWasTaken,1.362593e+09,None,None,t2_aubzb,True,False,1489072372,self.JUSTNOMIL,https://www.reddit.com/r/JUSTNOMIL/comments/5y...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,i_like_quesadillas,1.466648e+09,None,None,t2_yy3oq,False,False,1489259906,self.wallstreetbets,https://www.reddit.com/r/wallstreetbets/commen...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df['created_utc'].head()

0    1488982363
1    1488989677
2    1488996369
3    1489072372
4    1489259906
Name: created_utc, dtype: int64

In [10]:
def epoch_time(x):
    datetime_time = datetime.fromtimestamp(x)
    timestampStr = datetime_time.strftime("%b-%Y")
    return timestampStr


In [11]:
df['created_utc'].unique()

array([1488982363, 1488989677, 1488996369, ..., 1614900844, 1614902446,
       1614912207], dtype=int64)

In [12]:
df['created_utc'] = df['created_utc'].apply(lambda x: epoch_time(int(float(x))))

In [13]:
df_cutc = df['created_utc']
df_cutc.tail()

2044    Mar-2021
2045    Mar-2021
2046    Mar-2021
2047    Mar-2021
2048    Mar-2021
Name: created_utc, dtype: object

In [14]:
## Sub Reddit sentiment scores 


sentiments = []

for post in df['selftext']:
    try:
        text = post
        results = sid.polarity_scores(text)
        compound = results['compound']
        pos = results['pos']
        neu = results['neu']
        neg = results['neg']

        sentiments.append({
            'text': text,
            'Compound': compound,
            'Positive': pos,
            'Negative': neg,
            'Neutral': neu,
        })
    except AttributeError:
        pass
    
scores_df = pd.DataFrame(sentiments)
scores_df.head()

,text,Compound,Positive,Negative,Neutral
0,[removed],0.0000,0.000,0.000,1.000
1,Please post any and all gameday questions rega...,0.9971,0.081,0.025,0.894
2,"So long story short, I bought $JNUG last week ...",0.1431,0.063,0.056,0.881
3,Nancy was a SAHM for most of DH's childhood. ...,-0.9877,0.069,0.107,0.824
4,Weeks ago I saw numerous bitching posts about ...,0.9969,0.105,0.044,0.850
